In [11]:
import csv

In [12]:
with open("url/cache_2", mode="r", newline="", encoding="utf-8") as f:
    thutucs = [e for e in csv.DictReader(f)]
    thutuc_fields = list(thutucs[0].keys())

In [14]:
print(thutuc_fields)

['code', 'name', 'field', 'org', 'level', 'link', 'related', 'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Thời gian giải quyết', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 'Kết quả', 'Phí, lệ phí', 'Tên mẫu đơn, tờ khai', 'Yêu cầu, điều kiện', 'Căn cứ pháp lý']
